In [4]:
import uproot
import numpy as np
import os
import math
import progressbar

In [2]:
def prepareArray(origArray):
    preparedArray = np.zeros(len(origArray), dtype=complex)
    for i in range(0, len(origArray)):
        preparedArray[i] = origArray[i][0]
    return preparedArray

In [21]:
#ampPath = "/nfs/freenas/tuph/e18/project/compass/analysis/tkumschier/ROOTPWA/test/TEST_FOLDER/ROOTPWA_MC_TEST0/amps/"
#ampPath = "/nfs/freenas/tuph/e18/project/compass/analysis/tkumschier/ROOTPWA/test/ROOTPWA_MC_TEST_500/amps/"
ampPath = "/nfs/freenas/tuph/e18/project/compass/analysis/tkumschier/ROOTPWA/test/TEST_FOLDER_RUNS2/ROOTPWA_MC_TEST0/amps/"


onlyfiles = []
for filename in sorted(os.listdir(ampPath)):
    onlyfiles.append(filename)
nmbWaves = len(onlyfiles)
ampFile1 = uproot.open(ampPath + onlyfiles[0])
nmbEvents = len(ampFile1[ampFile1.keys()[0]]['amplitude'].get('_incohSubAmps._real').array())
amp = np.zeros((nmbWaves, nmbWaves), dtype = complex)
sd = np.zeros((nmbWaves, nmbWaves), dtype = complex)
ampMatrix = np.zeros((nmbWaves,nmbEvents), dtype = complex)
errorMatrix = np.zeros((nmbWaves,nmbEvents), dtype = complex)


bar = progressbar.ProgressBar(maxval=nmbWaves, widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])
bar.start()
for waveIndexI in range(0, nmbWaves):
    if ampMatrix[waveIndexI].all() == 0:
        ampFile1 = uproot.open(ampPath + onlyfiles[waveIndexI])
        key1 = ampFile1.keys()[0]

        incohSubAmpsReal1 = ampFile1[key1]['amplitude'].get('_incohSubAmps._real').array()
        incohSubAmpsImag1 = ampFile1[key1]['amplitude'].get('_incohSubAmps._imag').array()
        ampMatrix[waveIndexI] = prepareArray(np.array(incohSubAmpsReal1 + 1j * incohSubAmpsImag1))
        #print ampMatrix[waveIndexI]
        #raise ValueError('Exit.')


    bar.update(waveIndexI+1)
    
    for waveIndexJ in range(0, waveIndexI + 1): #nmbWaves):
        if ampMatrix[waveIndexJ].all() == 0:
            ampFile2 = uproot.open(ampPath + onlyfiles[waveIndexJ])
            key2 = ampFile2.keys()[0]

            incohSubAmpsReal2 = ampFile2[key2]['amplitude'].get('_incohSubAmps._real').array()
            incohSubAmpsImag2 = ampFile2[key2]['amplitude'].get('_incohSubAmps._imag').array()
            ampMatrix[waveIndexJ] = prepareArray(np.array(incohSubAmpsReal2 + 1j * incohSubAmpsImag2))

        integral = np.sum(ampMatrix[waveIndexI] * ampMatrix[waveIndexJ].conjugate())/nmbEvents
        amp[waveIndexI][waveIndexJ] = integral
        amp[waveIndexJ][waveIndexI] = integral     
        
        sdReal = sum(pow((ampMatrix[waveIndexI] * ampMatrix[waveIndexJ].conjugate()).real, 2)).real
        sdImag = sum(pow((ampMatrix[waveIndexI] * ampMatrix[waveIndexJ].conjugate()).imag, 2)).real
        #print "Var1:", sdReal
        
        
        '''
        sdReal = 0.
        sdImag = 0.
        for subAmp in range(0, len(incohSubAmpsComplex1)):
            sdReal += pow((incohSubAmpsComplex1[subAmp] * incohSubAmpsComplex2.conjugate()[subAmp]).real, 2)
            sdImag += pow((incohSubAmpsComplex1[subAmp] * incohSubAmpsComplex2.conjugate()[subAmp]).imag, 2)
            #print (incohSubAmpsComplex1 * incohSubAmpsComplex2.conjugate()).real[subAmp][0].real, (incohSubAmpsComplex1[subAmp][0] * incohSubAmpsComplex2.conjugate()[subAmp][0]).real
        print "Var2:", sdReal
      '''
    

        sdReal = math.sqrt(sdReal/float(nmbEvents) - pow(amp[waveIndexI][waveIndexJ].real,2))
        sdImag = math.sqrt(sdImag/float(nmbEvents) - pow(amp[waveIndexI][waveIndexJ].imag,2))
        sdComplex = complex(sdReal, sdImag)        
        errorComplex = complex(sdReal/math.sqrt(nmbEvents), sdImag/math.sqrt(nmbEvents))
        errorMatrix[waveIndexI][waveIndexJ] = errorComplex
        errorMatrix[waveIndexJ][waveIndexI] = errorComplex
        
        
        sd[waveIndexI][waveIndexJ] = sdComplex
        sd[waveIndexJ][waveIndexI] = sdComplex
 
        #print "%i, %i: %s, %s, %s" %(waveIndexI, waveIndexJ, amp[waveIndexI][waveIndexJ], sd[waveIndexI][waveIndexJ], errorComplex)
        #print "%i, %i: %s, %s" %(waveIndexI, waveIndexJ, amp[waveIndexI][waveIndexJ], errorComplex)
        #print np.cov(amp/nmbEvents)
            #raise ValueError('Exit.')


In [22]:
np.save('AlternativeBerechnung_OhneTest_50k.npy', np.array([(amp,errorMatrix)], dtype=[('amp', np.ndarray),('errorMatrix', np.ndarray)]))